In [1]:
import requests

import numpy as np
import pandas as pd

### Using TraderMade REST API for historical data

In [2]:
API_KEY = "api_key_here"

In [3]:
def request_pair(src:str, dst:str, start_date:str="2022-11-29 08:30", end_date:str="2022-11-29 09:00"):
    req = f"https://marketdata.tradermade.com/api/v1/tick_historical_sample/{src}{dst}/{start_date}/{end_date}?api_key={API_KEY}&format=json"
    resp = requests.get(req)
    return resp         

def parse_json(resp) -> pd.DataFrame:
    json_df = pd.DataFrame(resp.json())
    df = pd.DataFrame(json_df["quotes"].to_list())
    df[["from","to"]] = df["inst"].str.extract('(.{3,3})' * 2)
    df = df.drop(columns="inst")
    
    df["start_date"] = json_df["start_date"]
    df["end_date"] = json_df["end_date"]
    return df

def save_frame(df:pd.DataFrame, name:str) -> None:
    df.to_pickle(f"{name}.pickle")

def download(src, dst):
    try:
        resp = request_pair(src, dst)
        df = parse_json(resp)
        save_frame(df, f"{src}{dst}")
    except Exception as e:
        print("error")
        print(resp.json())

#download("GBP", "USD")

### Setup Data

In [7]:
pair_list = [ 
    ("BTC", "JPY"),
    ("BTC", "USD"),
    ("USD", "JPY"),
    ("ETH", "JPY"),
    ("ETH", "USD"),
    ("ETH", "BTC"),
]

nodes = list(set([x[0] for x in pair_list] + [x[1] for x in pair_list]))

dfs = []
for src, dst in pair_list:
    df = pd.read_pickle(f"\data\{src}{dst}.pickle")
    dfs.append(df)

FileNotFoundError: [Errno 2] No such file or directory: '\\data\\BTCJPY.pickle'

In [ ]:
combined = pd.concat(dfs)
combined = combined.sort_values(by="time")

combined

In [ ]:
## Bin ticks into timestamps, (the backtest will lose accuracy)
combined["bin time"] = pd.cut(combined["time"], int(combined.shape[0]/2))
combined_pt = combined.pivot_table(index="bin time", columns=["from", "to"], values=["bid", "ask"], aggfunc="mean")

### No binning done, (likely no arbs will be found)
#combined_pt = combined.pivot_table(index="time", columns=["from", "to"], values=["bid", "ask", "exch_rate_out", "exch_rate_in"], aggfunc="mean") # no timestamp bins

valid_periods = combined_pt[combined_pt.loc[:, (["ask","bid"], slice(None))].count(axis=1) > 1] # filter rows with <=1 nodes in graph, arb is obviously impossible 
valid_periods

### Bellman-Ford for cycle detection

Idea:
  - Model each tick as a graph of n nodes (assets), where each edge cost represents the current exchange rate
    - Ideally, we have a complete graph with nC2 edges, but there might be some lacking data
  - We define the cost of a path as multiplicative and not additive:
    - As such, we take the log(exchange rate) as our edge cost instead, since log(a) + log(b) = log(ab)
  - If there exists arbitrage in the market, then there exists a negative weighted cycle 
    - We use n passes of the Bellman-Ford algorithm to detect whether such a cycle exists
      - This takes O(VE) time, but since we have a (nearly) complete graph, it is O(V^3) time

In [ ]:
class Graph:
    def __init__(self, nodes:list):
        '''
        Graph DS, implemented as EdgeList
        '''
        self.nodes = nodes
        self.edges = []

    def add_edge(self, src, dst, cost) -> None:
        self.edges.append((src, dst, -np.log(cost)))

    def bellman_ford(self):
        dist = {node: np.inf for node in self.nodes}
        dist[self.nodes[0]] = 0

        parent = {node: -1 for node in self.nodes}

        for _ in range(0, len(self.nodes)-1):
            for u, v, cost in self.edges:
                #print(u, v, cost)
                if dist[v] > dist[u] + cost:
                    dist[v] = dist[u] + cost
                    parent[v] = u
        
        # check neg cycle
        C = None
        for u,v,cost in self.edges:
            if dist[v] > dist[u] + cost:
                C = v
                
        if C != None:
            # cycle
            for _ in range(len(self.nodes)):
                if parent[C] == -1:
                    return False
                C = parent[C]
            cycle = []
            v = C
            while True:
                cycle.append(v)
                if (v == C and len(cycle) > 1):
                    break
                else:
                    v = parent[v]
            cycle.reverse()

            return cycle
        else:
            return False 

In [ ]:
num_cycles = 0
for i, row in valid_periods.iterrows():
    G = Graph(nodes=nodes)
    
    ask = row["ask"]
    bid = row["bid"]
    for src, dst in ask.index:
        cost1 = bid[(src, dst)]
        cost2 = 1 / ask[(src, dst)]
        G.add_edge(src, dst, cost1)
        G.add_edge(dst, src, cost2)

    cycle = G.bellman_ford()
    if cycle:
        print(f"Negative weight cycle detected at bin time {i} | Cycle: {cycle}")
        prev = cycle[0]
        product = 1
        for curr in cycle[1:]:
            if (prev, curr) in bid.index:
                rate = bid[(prev, curr)]
                product *= rate
            else:
                rate = 1 / ask[(curr, prev)]
                product *= rate            
            prev = curr
        
        if product > 1:
            print(f"Return: {product}")
            num_cycles+=1

print(f"Num cycles detected = {num_cycles}")